In [6]:
# CELL 1: HỆ THỐNG ĐĂNG KÝ NGƯỜI DÙNG MỚI - DATABASE NÂNG CẤP
import cv2
import sqlite3
import pickle
import os
from deepface import DeepFace
import numpy as np
from datetime import datetime

# Khởi tạo cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# CẤU HÌNH DATABASE MỚI
DB_FILE = "face_system_v2.db"
KNOWN_FACES_DIR = "known_faces_v2"
os.makedirs(KNOWN_FACES_DIR, exist_ok=True)

def init_database():
    """Khởi tạo database SQLite mới"""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    
    # Bảng người dùng với nhiều thông tin hơn
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            full_name TEXT NOT NULL,
            birth_year INTEGER,
            gender TEXT,
            address TEXT,
            phone TEXT,
            email TEXT,
            note TEXT,
            created_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    # Bảng khuôn mặt với metadata
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS face_data (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER,
            embedding BLOB NOT NULL,
            image_path TEXT,
            image_size INTEGER,
            created_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (user_id) REFERENCES users (id) ON DELETE CASCADE
        )
    ''')
    
    # Index để tăng tốc độ tìm kiếm
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_user_id ON face_data(user_id)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_full_name ON users(full_name)')
    
    conn.commit()
    conn.close()
    print(" Database mới đã được khởi tạo")

def calculate_age(birth_year):
    """Tính tuổi từ năm sinh"""
    if not birth_year:
        return None
    current_year = datetime.now().year
    return current_year - birth_year

def add_user_to_database(full_name, birth_year=None, gender="", address="", phone="", email="", note=""):
    """Thêm người dùng mới vào database"""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    
    cursor.execute('''
        INSERT INTO users (full_name, birth_year, gender, address, phone, email, note)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (full_name, birth_year, gender, address, phone, email, note))
    
    user_id = cursor.lastrowid
    conn.commit()
    conn.close()
    
    age_text = f", {calculate_age(birth_year)} tuổi" if birth_year else ""
    gender_text = f", Giới tính: {gender}" if gender else ""
    print(f"✅ Đã thêm: {full_name}{age_text}{gender_text}")
    return user_id

def update_user_in_database(user_id, full_name, birth_year=None, gender="", address="", phone="", email="", note=""):
    """Cập nhật thông tin người dùng"""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    
    cursor.execute('''
        UPDATE users 
        SET full_name = ?, birth_year = ?, gender = ?, address = ?, phone = ?, email = ?, note = ?, updated_date = CURRENT_TIMESTAMP
        WHERE id = ?
    ''', (full_name, birth_year, gender, address, phone, email, note, user_id))
    
    conn.commit()
    conn.close()
    
    age_text = f", {calculate_age(birth_year)} tuổi" if birth_year else ""
    print(f"✅ Đã cập nhật: {full_name}{age_text}")

def delete_user_from_database(user_id):
    """Xóa người dùng khỏi database"""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    
    # Lấy thông tin trước khi xóa
    cursor.execute('SELECT full_name FROM users WHERE id = ?', (user_id,))
    result = cursor.fetchone()
    full_name = result[0] if result else "Unknown"
    
    # Xóa các face data trước (CASCADE sẽ tự động xóa)
    cursor.execute('DELETE FROM face_data WHERE user_id = ?', (user_id,))
    
    # Xóa người dùng
    cursor.execute('DELETE FROM users WHERE id = ?', (user_id,))
    
    conn.commit()
    conn.close()
    
    # Xóa ảnh trong thư mục
    import glob
    pattern = f"{KNOWN_FACES_DIR}/{full_name}_*.jpg"
    for file_path in glob.glob(pattern):
        try:
            os.remove(file_path)
        except:
            pass
    
    print(f"🗑️ Đã xóa: {full_name}")

def add_face_data(user_id, embedding, image_path):
    """Thêm dữ liệu khuôn mặt vào database"""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    
    embedding_blob = pickle.dumps(embedding)
    image_size = os.path.getsize(image_path) if os.path.exists(image_path) else 0
    
    cursor.execute('''
        INSERT INTO face_data (user_id, embedding, image_path, image_size)
        VALUES (?, ?, ?, ?)
    ''', (user_id, embedding_blob, image_path, image_size))
    
    conn.commit()
    conn.close()

def get_face_embedding(face_image):
    """Lấy embedding từ khuôn mặt"""
    try:
        embedding_data = DeepFace.represent(
            face_image,
            model_name="Facenet",
            enforce_detection=False
        )
        
        if isinstance(embedding_data, list):
            return embedding_data[0]['embedding']
        else:
            return embedding_data['embedding']
    except Exception as e:
        print(f" Lỗi lấy embedding: {e}")
        return None

def register_new_user():
    """Đăng ký người dùng mới"""
    print("\n" + "="*60)
    print("📸 ĐĂNG KÝ NGƯỜI DÙNG MỚI")
    print("="*60)
    
    # Nhập thông tin
    print("\n📝 NHẬP THÔNG TIN:")
    full_name = input("Họ và tên (có dấu): ").strip()
    if not full_name:
        print(" Họ tên không được để trống!")
        return False
    
    # Nhập năm sinh
    birth_year = None
    while True:
        birth_year_input = input("Năm sinh (Enter để bỏ qua): ").strip()
        if not birth_year_input:
            break
        try:
            birth_year = int(birth_year_input)
            current_year = datetime.now().year
            if birth_year < 1900 or birth_year > current_year:
                print(f"LƯU Ý: Năm sinh phải từ 1900 đến {current_year}")
                continue
            age = calculate_age(birth_year)
            print(f"   ✅ Tuổi: {age}")
            break
        except ValueError:
            print("LƯU Ý: Năm sinh phải là số!")
    
    # Nhập giới tính
    gender = ""
    gender_input = input("Giới tính (Nam/Nữ/Enter để bỏ qua): ").strip()
    if gender_input.lower() in ['nam', 'nữ']:
        gender = gender_input
    
    address = input("Địa chỉ: ").strip()
    phone = input("Số điện thoại: ").strip()
    email = input("Email: ").strip()
    note = input("Ghi chú: ").strip()
    
    # Hiển thị thông tin đã nhập
    print(f"\n👤 THÔNG TIN ĐÃ NHẬP:")
    print(f"   Họ tên: {full_name}")
    if birth_year:
        print(f"   Năm sinh: {birth_year} ({calculate_age(birth_year)} tuổi)")
    if gender:
        print(f"   Giới tính: {gender}")
    if address:
        print(f"   Địa chỉ: {address}")
    if phone:
        print(f"   SĐT: {phone}")
    if email:
        print(f"   Email: {email}")
    if note:
        print(f"   Ghi chú: {note}")
    
    confirm = input("\n✅ Xác nhận đăng ký? (y/n): ").lower()
    if confirm != 'y':
        print("❌ Hủy đăng ký!")
        return False
    
    # Chụp ảnh khuôn mặt
    print("\n📷 Đang mở camera...")
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("❌ Không thể mở camera!")
        return False
    
    print(" Hướng dẫn chụp ảnh:")
    print("   - Giữ khuôn mặt trong khung xanh")
    print("   - Nhấn 'c' để chụp ảnh")
    print("   - Nhấn 'q' để hủy")
    print("   - Cần chụp 3 ảnh mẫu")
    
    captured_faces = 0
    max_faces = 3
    
    while captured_faces < max_faces:
        ret, frame = cap.read()
        if not ret:
            print("LƯU Ý: Lỗi camera!")
            break
            
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 5, minSize=(100, 100))
        
        display_frame = frame.copy()
        
        if len(faces) > 0:
            x, y, w, h = faces[0]
            cv2.rectangle(display_frame, (x, y), (x+w, y+h), (0, 255, 0), 3)
            cv2.putText(display_frame, f"Picture {captured_faces + 1}/{max_faces} - click 'C' to  take a photo", 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        cv2.imshow(f"ĐĂNG KÝ: {full_name}", display_frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('c') and len(faces) > 0:
            x, y, w, h = faces[0]
            face_roi = frame[y:y+h, x:x+w]
            face_resized = cv2.resize(face_roi, (224, 224))
            
            embedding = get_face_embedding(face_resized)
            if embedding is not None:
                if captured_faces == 0:
                    user_id = add_user_to_database(full_name, birth_year, gender, address, phone, email, note)
                
                image_path = f"{KNOWN_FACES_DIR}/{full_name}_{datetime.now().strftime('%H%M%S')}.jpg"
                cv2.imwrite(image_path, face_resized)
                
                add_face_data(user_id, embedding, image_path)
                
                captured_faces += 1
                print(f"✅ Đã chụp ảnh {captured_faces}/{max_faces}")
                
                if captured_faces >= max_faces:
                    print(f"🎉 Đăng ký thành công: {full_name}")
                    break
            else:
                print("LƯU Ý: Không thể trích xuất đặc trưng khuôn mặt")
                
        elif key == ord('q'):
            print(" Hủy đăng ký!")
            break
    
    cap.release()
    cv2.destroyAllWindows()
    return captured_faces > 0

def edit_user():
    """Sửa thông tin người dùng"""
    view_database()
    
    try:
        user_id = int(input("\nNhập ID người cần sửa: ").strip())
    except:
        print("LƯU Ý: ID không hợp lệ!")
        return
    
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM users WHERE id = ?', (user_id,))
    user = cursor.fetchone()
    conn.close()
    
    if not user:
        print("LƯU Ý: Không tìm thấy người dùng!")
        return
    
    print(f"\n📝 THÔNG TIN HIỆN TẠI:")
    print(f"   Họ tên: {user[1]}")
    print(f"   Năm sinh: {user[2] or 'Chưa có'}")
    print(f"   Giới tính: {user[3] or 'Chưa có'}")
    print(f"   Địa chỉ: {user[4] or 'Chưa có'}")
    print(f"   SĐT: {user[5] or 'Chưa có'}")
    print(f"   Email: {user[6] or 'Chưa có'}")
    print(f"   Ghi chú: {user[7] or 'Chưa có'}")
    
    print("\n📝 NHẬP THÔNG TIN MỚI (Enter để giữ nguyên):")
    new_full_name = input(f"Họ tên [{user[1]}]: ").strip() or user[1]
    
    new_birth_year = user[2]
    birth_year_input = input(f"Năm sinh [{user[2] or 'Chưa có'}]: ").strip()
    if birth_year_input:
        try:
            new_birth_year = int(birth_year_input)
            current_year = datetime.now().year
            if new_birth_year < 1900 or new_birth_year > current_year:
                print(f"LƯU Ý: Năm sinh phải từ 1900 đến {current_year}")
                return
        except ValueError:
            print("LƯU Ý: Năm sinh phải là số!")
            return
    
    new_gender = input(f"Giới tính [{user[3] or 'Chưa có'}]: ").strip() or user[3]
    new_address = input(f"Địa chỉ [{user[4] or 'Chưa có'}]: ").strip() or user[4]
    new_phone = input(f"SĐT [{user[5] or 'Chưa có'}]: ").strip() or user[5]
    new_email = input(f"Email [{user[6] or 'Chưa có'}]: ").strip() or user[6]
    new_note = input(f"Ghi chú [{user[7] or 'Chưa có'}]: ").strip() or user[7]
    
    update_user_in_database(user_id, new_full_name, new_birth_year, new_gender, new_address, new_phone, new_email, new_note)

def delete_user():
    """Xóa người dùng"""
    view_database()
    
    try:
        user_id = int(input("\nNhập ID người cần xóa: ").strip())
    except:
        print("LƯU Ý: ID không hợp lệ!")
        return
    
    confirm = input("⚠️  CHẮC CHẮN MUỐN XÓA? (y/n): ").lower()
    if confirm == 'y':
        delete_user_from_database(user_id)
    else:
        print("LƯU Ý: Đã hủy xóa!")

def view_database():
    """Xem toàn bộ database"""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    
    cursor.execute('''
        SELECT u.id, u.full_name, u.birth_year, u.gender, u.address, u.phone, u.email, 
               u.note, u.created_date, COUNT(fd.id) as face_count
        FROM users u
        LEFT JOIN face_data fd ON u.id = fd.user_id
        GROUP BY u.id
        ORDER BY u.created_date DESC
    ''')
    
    results = cursor.fetchall()
    conn.close()
    
    print("\n DANH SÁCH NGƯỜI DÙNG:")
    print("=" * 100)
    if not results:
        print("   Database trống!")
    else:
        for row in results:
            user_id, full_name, birth_year, gender, address, phone, email, note, created_date, face_count = row
            age_text = f", {calculate_age(birth_year)} tuổi" if birth_year else ""
            gender_text = f", {gender}" if gender else ""
            print(f"ID: {user_id} | {full_name}{age_text}{gender_text}")
            if address:
                print(f"    📍 {address}")
            if phone:
                print(f"    📞 {phone}")
            if email:
                print(f"    📧 {email}")
            if note:
                print(f"    📝 {note}")
            print(f"    🗓️  {created_date[:10]} | 📸 {face_count} ảnh mẫu")
            print("-" * 100)

def database_stats():
    """Thống kê database"""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    
    cursor.execute('SELECT COUNT(*) FROM users')
    user_count = cursor.fetchone()[0]
    
    cursor.execute('SELECT COUNT(*) FROM face_data')
    face_count = cursor.fetchone()[0]
    
    cursor.execute('SELECT COUNT(*) FROM users WHERE birth_year IS NOT NULL')
    has_birth_year = cursor.fetchone()[0]
    
    conn.close()
    
    print(f"\n📈 THỐNG KÊ DATABASE:")
    print(f"   👥 Số người dùng: {user_count}")
    print(f"   📸 Số ảnh mẫu: {face_count}")
    print(f"   🎂 Có năm sinh: {has_birth_year}")
    if user_count > 0:
        print(f"   📊 Số ảnh/người: {face_count/user_count:.1f}")

# KHỞI CHẠY
init_database()

print(" HỆ THỐNG QUẢN LÝ NGƯỜI DÙNG - DATABASE V2")
print("=" * 50)
print("1. Đăng ký người dùng mới")
print("2. Sửa thông tin người dùng")
print("3. Xóa người dùng")
print("4. Xem database")
print("5. Thống kê")
print("6. Thoát")

while True:
    choice = input("\nChọn chức năng (1-6): ").strip()
    
    if choice == '1':
        register_new_user()
    elif choice == '2':
        edit_user()
    elif choice == '3':
        delete_user()
    elif choice == '4':
        view_database()
    elif choice == '5':
        database_stats()
    elif choice == '6':
        print("👋 Đã thoát!")
        break
    else:
        print("LƯU Ý: Lựa chọn không hợp lệ!")


 Database mới đã được khởi tạo
 HỆ THỐNG QUẢN LÝ NGƯỜI DÙNG - DATABASE V2
1. Đăng ký người dùng mới
2. Sửa thông tin người dùng
3. Xóa người dùng
4. Xem database
5. Thống kê
6. Thoát

📸 ĐĂNG KÝ NGƯỜI DÙNG MỚI

📝 NHẬP THÔNG TIN:
   ✅ Tuổi: 20

👤 THÔNG TIN ĐÃ NHẬP:
   Họ tên: vÕ ĐẠI PHÁT
   Năm sinh: 2005 (20 tuổi)
   Giới tính: NAM

📷 Đang mở camera...
 Hướng dẫn chụp ảnh:
   - Giữ khuôn mặt trong khung xanh
   - Nhấn 'c' để chụp ảnh
   - Nhấn 'q' để hủy
   - Cần chụp 3 ảnh mẫu
✅ Đã thêm: vÕ ĐẠI PHÁT, 20 tuổi, Giới tính: NAM
✅ Đã chụp ảnh 1/3
✅ Đã chụp ảnh 2/3
✅ Đã chụp ảnh 3/3
🎉 Đăng ký thành công: vÕ ĐẠI PHÁT

 DANH SÁCH NGƯỜI DÙNG:
ID: 9 | vÕ ĐẠI PHÁT, 20 tuổi, NAM
    🗓️  2025-11-13 | 📸 3 ảnh mẫu
----------------------------------------------------------------------------------------------------
👋 Đã thoát!


In [1]:
!pip install detection_history

ERROR: Could not find a version that satisfies the requirement detection_history (from versions: none)
ERROR: No matching distribution found for detection_history


In [ ]:
# CELL 2: HỆ THỐNG NHẬN DIỆN KHUÔN MẶT - DATABASE MỚI
import cv2
from deepface import DeepFace
import threading
import numpy as np
from collections import OrderedDict
import sqlite3
import pickle
from datetime import datetime

# ===== PHẦN LƯU LỊCH SỬ =====
DB_FILE = "face_system_v2.db"

def init_history_db():
    """Khởi tạo bảng lịch sử"""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS detection_history (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            type TEXT NOT NULL,
            gender TEXT,
            emotion TEXT,
            age INTEGER,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    conn.commit()
    conn.close()

init_history_db()

last_save = {}
save_lock = threading.Lock()

def save_detection(result):
    """Lưu kết quả phát hiện vào database"""
    try:
        if result["status"] == "known":
            name = result['full_name']
            detection_type = "KNOWN"
        else:
            name = "UNKNOWN"
            detection_type = "UNKNOWN"
        
        gender = result.get('detected_gender', 'Unknown')
        emotion = result.get('emotion', 'Unknown')
        age = result.get('detected_age')
        
        key = f"{name}_{gender}"
        now = datetime.now()
        
        with save_lock:
            if key in last_save and (now - last_save[key]).seconds < 5:
                return
            last_save[key] = now
        
        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()
        cursor.execute('INSERT INTO detection_history (name, type, gender, emotion, age) VALUES (?, ?, ?, ?, ?)',
                      (name, detection_type, gender, emotion, age))
        conn.commit()
        conn.close()
        
        print(f"💾 Đã lưu: {name} | {gender} | {emotion} | {age} tuổi")
        
    except Exception as e:
        print(f"❌ Lỗi lưu: {e}")
# ===== KẾT THÚC PHẦN LƯU LỊCH SỬ =====

# Khởi tạo cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# 🔧 LỰA CHỌN CAMERA
def select_camera():
    """Menu lựa chọn camera"""
    print("\n🎥 LỰA CHỌN CAMERA:")
    print("1. 📷 Webcam Laptop (Mặc định)")
    print("2. 📷 Webcam USB 2")
    print("3. 📱 Điện thoại Android (IP: 10.240.60.67:8080)")
    print("4. 📱 Điện thoại iPhone (IP: 192.168.1.100:4747)")
    print("5. 🔧 Nhập URL tùy chỉnh")
    
    choice = input("Chọn camera (1-5): ").strip()
    
    if choice == '1':
        return 0
    elif choice == '2':
        return 1
    elif choice == '3':
        return "http://192.168.2.105:8080/video"
    elif choice == '4':
        return "http://192.168.2.105:8080/video"
    elif choice == '5':
        custom_url = input("Nhập URL camera: ").strip()
        return custom_url if custom_url else 0
    else:
        print("❌ Lựa chọn không hợp lệ! Dùng webcam mặc định.")
        return 0

def connect_camera(camera_source):
    """Kết nối camera"""
    print(f"🔗 Đang kết nối camera...")
    
    try:
        cap = cv2.VideoCapture(camera_source)
        
        if isinstance(camera_source, str) and camera_source.startswith('http'):
            cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
            cap.set(cv2.CAP_PROP_FPS, 15)
        
        ret, frame = cap.read()
        if ret and frame is not None:
            source_type = "Camera IP" if isinstance(camera_source, str) else f"Webcam {camera_source}"
            print(f"✅ Kết nối thành công: {source_type}")
            print(f"   📏 Kích thước: {frame.shape[1]}x{frame.shape[0]}")
            return cap
        else:
            cap.release()
            print("LƯU Ý: Không thể đọc frame từ camera này!")
            return None
    except Exception as e:
        print(f"LƯU Ý: Lỗi kết nối: {e}")
        return None

# Kết nối camera
camera_source = select_camera()
cap = connect_camera(camera_source)
if cap is None:
    print("🔄 Thử kết nối webcam mặc định...")
    cap = connect_camera(0)

if cap is None:
    print("LƯU Ý: KHÔNG THỂ KẾT NỐI CAMERA NÀO!")
    exit()

# Biến toàn cục
face_results = OrderedDict()
lock = threading.Lock()
max_concurrent_threads = 2
current_threads = 0
thread_lock = threading.Lock()

def get_all_face_data():
    """Lấy tất cả dữ liệu từ database mới"""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    
    cursor.execute('''
        SELECT u.id, u.full_name, u.birth_year, u.gender, u.address, u.phone, u.email, fd.embedding
        FROM users u
        JOIN face_data fd ON u.id = fd.user_id
    ''')
    
    results = cursor.fetchall()
    conn.close()
    
    faces_data = []
    for row in results:
        user_id, full_name, birth_year, gender, address, phone, email, embedding_blob = row
        embedding = pickle.loads(embedding_blob)
        
        age = None
        if birth_year:
            current_year = datetime.now().year
            age = current_year - birth_year
        
        faces_data.append({
            'user_id': user_id,
            'full_name': full_name,
            'age': age,
            'gender': gender,
            'address': address,
            'phone': phone,
            'email': email,
            'embedding': embedding
        })
    
    return faces_data

def recognize_face(face_image):
    """Nhận diện khuôn mặt"""
    try:
        current_embedding_data = DeepFace.represent(
            face_image,
            model_name="Facenet",
            enforce_detection=False
        )
        
        if isinstance(current_embedding_data, list):
            current_embedding = current_embedding_data[0]['embedding']
        else:
            current_embedding = current_embedding_data['embedding']
        
        known_faces = get_all_face_data()
        
        if not known_faces:
            return {"status": "unknown", "full_name": "", "age": None, "gender": "", "address": ""}, 0
        
        best_match = None
        best_similarity = 0
        similarity_threshold = 0.6
        
        for face_data in known_faces:
            stored_embedding = face_data['embedding']
            
            similarity = np.dot(current_embedding, stored_embedding) / (
                np.linalg.norm(current_embedding) * np.linalg.norm(stored_embedding)
            )
            
            if similarity > best_similarity and similarity > similarity_threshold:
                best_similarity = similarity
                best_match = face_data
        
        if best_match:
            person_info = {
                "status": "known",
                "user_id": best_match['user_id'],
                "full_name": best_match['full_name'],
                "age": best_match['age'],
                "gender": best_match['gender'],
                "address": best_match['address'],
                "phone": best_match['phone'],
                "email": best_match['email']
            }
            return person_info, best_similarity
        else:
            return {"status": "unknown", "full_name": "", "age": None, "gender": "", "address": ""}, 0
            
    except Exception as e:
        print(f"LƯU Ý: Lỗi nhận diện: {e}")
        return {"status": "error", "full_name": "", "age": None, "gender": "", "address": ""}, 0

def get_gender_from_analysis(analysis):
    """Lấy giới tính từ phân tích"""
    try:
        if isinstance(analysis, list):
            analysis = analysis[0]
            
        if 'gender' in analysis and isinstance(analysis['gender'], dict):
            man_score = analysis['gender'].get('Man', 0)
            woman_score = analysis['gender'].get('Woman', 0)
            
            if man_score > woman_score:
                return "Nam", man_score
            else:
                return "Nữ", woman_score
                
        return "Đang xử lý...", 0
                
    except Exception as e:
        return "Lỗi", 0

def analyze_face(face_roi, face_rect, face_id):
    """Phân tích khuôn mặt"""
    global face_results, current_threads
    
    try:
        person_info, similarity = recognize_face(face_roi)
        
        analysis = DeepFace.analyze(
            face_roi,
            actions=["emotion", "age", "gender"],
            enforce_detection=False,
            detector_backend="opencv",
            silent=True
        )
        
        gender_result, gender_score = get_gender_from_analysis(analysis)
        
        if isinstance(analysis, list):
            analysis = analysis[0]
        
        with lock:
            face_results[face_id] = {
                **person_info,
                "similarity": similarity,
                "emotion": analysis.get("dominant_emotion", "Unknown"),
                "detected_age": analysis.get("age", "Unknown"),
                "detected_gender": gender_result,
                "gender_score": gender_score,
                "rect": face_rect
            }
            
        # LƯU LỊCH SỬ PHÁT HIỆN
        save_detection(face_results[face_id])
            
        if person_info["status"] == "known":
            age_info = f", {person_info['age']} tuổi" if person_info['age'] else ""
            print(f"✅ NHẬN DIỆN: {person_info['full_name']}{age_info} - Độ tin cậy: {similarity:.1%}")
        else:
            print(f"👤 Người lạ - Cảm xúc: {analysis.get('dominant_emotion')}, Tuổi: {analysis.get('age')}, Giới tính: {gender_result}")

    except Exception as e:
        print(f"LƯU Ý: Lỗi phân tích {face_id}: {e}")
        with lock:
            face_results[face_id] = {
                "status": "error",
                "full_name": "",
                "age": None,
                "gender": "",
                "address": "",
                "similarity": 0,
                "emotion": "Lỗi",
                "detected_age": "Lỗi", 
                "detected_gender": "Lỗi",
                "gender_score": 0,
                "rect": face_rect
            }
    
    finally:
        with thread_lock:
            current_threads -= 1

def draw_text_with_background(img, text, position, font_scale, color, thickness=2, bg_color=(0, 0, 0)):
    """Vẽ chữ với nền đen"""
    x, y = position
    (text_width, text_height), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)
    
    cv2.rectangle(img, (x, y - text_height - 5), (x + text_width, y + 5), bg_color, -1)
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, thickness)
    
    return text_height + 10

# Tải database
known_faces = get_all_face_data()
known_faces_count = len(known_faces)

print("\n🚀 HỆ THỐNG NHẬN DIỆN KHUÔN MẶT - DATABASE V2")
print("=" * 60)
print(f"📊 Database: {known_faces_count} khuôn mặt")
print("LƯU Ý: Nhấn 'q' để thoát | 'c' để đổi camera")

frame_count = 0
analysis_interval = 40

while True:
    ret, frame = cap.read()
    if not ret:
        print("LƯU Ý: Mất kết nối camera...")
        break

    display_frame = frame.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, 1.1, 5, minSize=(80, 80))
    current_face_ids = set()
    
    for i, (x, y, w, h) in enumerate(faces):
        face_id = f"face_{i+1}"
        current_face_ids.add(face_id)
        
        # Màu sắc cho từng loại
        color = (255, 255, 0)  # Mặc định: Vàng (đang xử lý)
        
        with lock:
            if face_id in face_results:
                result = face_results[face_id]
                if result["status"] == "known":
                    color = (0, 255, 0)  # Xanh lá cho người quen
                elif result["status"] == "unknown":
                    color = (0, 0, 255)  # Đỏ cho người lạ
                elif result["status"] == "error":
                    color = (255, 0, 255)  # Tím cho lỗi
        
        # Vẽ khung khuôn mặt
        cv2.rectangle(display_frame, (x, y), (x+w, y+h), color, 3)
        cv2.putText(display_frame, f"#{i+1}", (x, y-15), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
        
        if w > 70 and h > 70:
            face_roi = frame[y:y+h, x:x+w]
            face_resized = cv2.resize(face_roi, (224, 224))
            
            with lock:
                needs_analysis = (face_id not in face_results or frame_count % analysis_interval == 0)
            
            if needs_analysis:
                with thread_lock:
                    if current_threads < max_concurrent_threads:
                        current_threads += 1
                        threading.Thread(
                            target=analyze_face, 
                            args=(face_resized, (x, y, w, h), face_id),
                            daemon=True
                        ).start()
        
        # HIỂN THỊ THÔNG TIN CHI TIẾT
        with lock:
            if face_id in face_results:
                result = face_results[face_id]
                
                info_lines = []
                
                if result["status"] == "known":
                    # NGƯỜI QUEN: Hiển thị thông tin từ database + dự đoán
                    info_lines.extend([
                        f" TEN: {result['full_name']}",
                        f" TUOI: {result['age'] or result['detected_age']}",
                        f" GIOI TINH: {result['gender'] or result['detected_gender']}",
                        f" CAM XUC: {result['emotion']}",
                        f" DIA CHI: {result['address']}",
                        f" SDT: {result['phone']}",
                        f" TIN CAY: {result['similarity']:.1%}"
                    ])
                    
                    # Loại bỏ các dòng trống
                    info_lines = [line for line in info_lines if not line.endswith(': ')]
                    
                elif result["status"] == "unknown":
                    # NGƯỜI LẠ: Chỉ hiển thị dự đoán
                    info_lines.extend([
                        f" TRANG THAI: NGUOI LA",
                        f" TUOI: {result['detected_age']}",
                        f" GIOI TINH: {result['detected_gender']}",
                        f" CAM XUC: {result['emotion']}",
                        f" DO TIN CAY: {result['gender_score']:.1f}%"
                    ])
                else:
                    # LỖI
                    info_lines.extend([
                        f"❌ LOI PHAN TICH",
                        f"Vui long thu lai"
                    ])
                
                # Hiển thị thông tin bên cạnh khuôn mặt
                text_x = x + w + 10
                text_y = y + 25
                
                for j, text in enumerate(info_lines):
                    y_pos = text_y + (j * 25)
                    text_height = draw_text_with_background(display_frame, text, (text_x, y_pos), 0.5, color, 1)
                    
                    # Giới hạn số dòng hiển thị để không tràn màn hình
                    if j >= 8:  # Tối đa 8 dòng
                        break

    with lock:
        expired_faces = [fid for fid in face_results if fid not in current_face_ids]
        for fid in expired_faces:
            del face_results[fid]

    frame_count += 1

    # Thống kê
    known_faces_detected = 0
    unknown_faces_detected = 0
    with lock:
        for result in face_results.values():
            if result["status"] == "known":
                known_faces_detected += 1
            elif result["status"] == "unknown":
                unknown_faces_detected += 1
    
    stats_text = f"PHAT HIEN: {len(faces)} | QUEN: {known_faces_detected} | LA: {unknown_faces_detected}"
    draw_text_with_background(display_frame, stats_text, (10, 40), 0.7, (255, 255, 255), 2)
    
    # Hiển thị hướng dẫn
    help_text = "Q: Thoat | C: Doi camera"
    draw_text_with_background(display_frame, help_text, (10, 70), 0.6, (255, 255, 255), 1)

    cv2.imshow("🤖 HE THONG NHAN DIEN KHUON MAT - V2", display_frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('c'):
        print("🔄 Đang chuyển camera...")
        cap.release()
        camera_source = select_camera()
        cap = connect_camera(camera_source)

cap.release()
cv2.destroyAllWindows()
print("👋 Đã thoát!")


🎥 LỰA CHỌN CAMERA:
1. 📷 Webcam Laptop (Mặc định)
2. 📷 Webcam USB 2
3. 📱 Điện thoại Android (IP: 10.240.60.67:8080)
4. 📱 Điện thoại iPhone (IP: 192.168.1.100:4747)
5. 🔧 Nhập URL tùy chỉnh
🔗 Đang kết nối camera...
✅ Kết nối thành công: Webcam 0
   📏 Kích thước: 640x480

🚀 HỆ THỐNG NHẬN DIỆN KHUÔN MẶT - DATABASE V2
📊 Database: 3 khuôn mặt
LƯU Ý: Nhấn 'q' để thoát | 'c' để đổi camera
💾 Đã lưu: vÕ ĐẠI PHÁT | Nam | fear | 29 tuổi
✅ NHẬN DIỆN: vÕ ĐẠI PHÁT, 20 tuổi - Độ tin cậy: 86.7%
✅ NHẬN DIỆN: vÕ ĐẠI PHÁT, 20 tuổi - Độ tin cậy: 86.7%
✅ NHẬN DIỆN: vÕ ĐẠI PHÁT, 20 tuổi - Độ tin cậy: 90.3%
✅ NHẬN DIỆN: vÕ ĐẠI PHÁT, 20 tuổi - Độ tin cậy: 90.3%
✅ NHẬN DIỆN: vÕ ĐẠI PHÁT, 20 tuổi - Độ tin cậy: 91.3%
✅ NHẬN DIỆN: vÕ ĐẠI PHÁT, 20 tuổi - Độ tin cậy: 81.2%
💾 Đã lưu: vÕ ĐẠI PHÁT | Nam | neutral | 26 tuổi
✅ NHẬN DIỆN: vÕ ĐẠI PHÁT, 20 tuổi - Độ tin cậy: 86.8%
✅ NHẬN DIỆN: vÕ ĐẠI PHÁT, 20 tuổi - Độ tin cậy: 86.8%
✅ NHẬN DIỆN: vÕ ĐẠI PHÁT, 20 tuổi - Độ tin cậy: 91.4%
✅ NHẬN DIỆN: vÕ ĐẠI PHÁT, 20 tuổi - Đ